In [14]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from openai import OpenAI
from dotenv import load_dotenv
import os
import sqlite3
import pandas as pd

In [9]:
openai = OpenAI(
    api_key= "-TjFCsia_r7.oivr6in~4ZJ", # Refer to Create a secret key section
    base_url="https://cloud.olakrutrim.com/v1",
)

In [17]:
db_conn = sqlite3.connect("chinook.db")
result = pd.read_sql_query("SELECT type, sql FROM sqlite_master WHERE sql is not null", db_conn)

In [20]:
db = FAISS.from_texts(result['sql'].to_list(), OllamaEmbeddings(model="nomic-embed-text"))

In [18]:
for ddl in result['sql'].to_list():
  #train faiss db here
  pass


In [21]:
golden = db.as_retriever()

In [23]:
golden.invoke("Which tracks were purchased in the order with maximum tracks purchased?")

[Document(page_content='CREATE INDEX [IFK_TrackAlbumId] ON "tracks" ([AlbumId])'),
 Document(page_content='CREATE INDEX [IFK_PlaylistTrackTrackId] ON "playlist_track" ([TrackId])'),
 Document(page_content='CREATE INDEX [IFK_TrackGenreId] ON "tracks" ([GenreId])'),
 Document(page_content='CREATE INDEX [IFK_InvoiceLineTrackId] ON "invoice_items" ([TrackId])')]

In [27]:
embedder = OllamaEmbeddings(model="nomic-embed-text")
q = embedder.embed_query("Which tracks were purchased in the order with maximum tracks purchased?")
a = db.similarity_search_by_vector(q, k=10)

In [26]:
def combineDocs(docs):
    context = "\n\n".join(f'\n\n{doc.page_content}\n\n' for doc in docs)
    return context

In [28]:
combineDocs(a)

'\n\nCREATE INDEX [IFK_TrackAlbumId] ON "tracks" ([AlbumId])\n\n\n\n\n\nCREATE INDEX [IFK_PlaylistTrackTrackId] ON "playlist_track" ([TrackId])\n\n\n\n\n\nCREATE INDEX [IFK_TrackGenreId] ON "tracks" ([GenreId])\n\n\n\n\n\nCREATE INDEX [IFK_InvoiceLineTrackId] ON "invoice_items" ([TrackId])\n\n\n\n\n\nCREATE INDEX [IFK_TrackMediaTypeId] ON "tracks" ([MediaTypeId])\n\n\n\n\n\nCREATE TABLE "playlists"\r\n(\r\n    [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(120)\r\n)\n\n\n\n\n\nCREATE TABLE "playlist_track"\r\n(\r\n    [PlaylistId] INTEGER  NOT NULL,\r\n    [TrackId] INTEGER  NOT NULL,\r\n    CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY  ([PlaylistId], [TrackId]),\r\n    FOREIGN KEY ([PlaylistId]) REFERENCES "playlists" ([PlaylistId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\r\n    FOREIGN KEY ([TrackId]) REFERENCES "tracks" ([TrackId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\n\n\n\n\nCREATE INDEX [IFK_AlbumArtistId] ON "albums" ([Art

In [31]:
chat_completion = openai.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct",
    messages=[
    {"role": "system", "content": """'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. '\n\nCREATE INDEX [IFK_TrackAlbumId] ON "tracks" ([AlbumId])\n\n\n\n\n\nCREATE INDEX [IFK_PlaylistTrackTrackId] ON "playlist_track" ([TrackId])\n\n\n\n\n\nCREATE INDEX [IFK_TrackGenreId] ON "tracks" ([GenreId])\n\n\n\n\n\nCREATE INDEX [IFK_InvoiceLineTrackId] ON "invoice_items" ([TrackId])\n\n\n\n\n\nCREATE INDEX [IFK_TrackMediaTypeId] ON "tracks" ([MediaTypeId])\n\n\n\n\n\nCREATE TABLE "playlists"\r\n(\r\n    [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(120)\r\n)\n\n\n\n\n\nCREATE TABLE "playlist_track"\r\n(\r\n    [PlaylistId] INTEGER  NOT NULL,\r\n    [TrackId] INTEGER  NOT NULL,\r\n    CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY  ([PlaylistId], [TrackId]),\r\n    FOREIGN KEY ([PlaylistId]) REFERENCES "playlists" ([PlaylistId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\r\n    FOREIGN KEY ([TrackId]) REFERENCES "tracks" ([TrackId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\n\n\n\n\nCREATE INDEX [IFK_AlbumArtistId] ON "albums" ([ArtistId])\n\n\n\n\n\nCREATE TABLE "tracks"\r\n(\r\n    [TrackId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(200)  NOT NULL,\r\n    [AlbumId] INTEGER,\r\n    [MediaTypeId] INTEGER  NOT NULL,\r\n    [GenreId] INTEGER,\r\n    [Composer] NVARCHAR(220),\r\n    [Milliseconds] INTEGER  NOT NULL,\r\n    [Bytes] INTEGER,\r\n    [UnitPrice] NUMERIC(10,2)  NOT NULL,\r\n    FOREIGN KEY ([AlbumId]) REFERENCES "albums" ([AlbumId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\r\n    FOREIGN KEY ([GenreId]) REFERENCES "genres" ([GenreId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION,\r\n    FOREIGN KEY ([MediaTypeId]) REFERENCES "media_types" ([MediaTypeId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\n\n\n\n\nCREATE TABLE "albums"\r\n(\r\n    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Title] NVARCHAR(160)  NOT NULL,\r\n    [ArtistId] INTEGER  NOT NULL,\r\n    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)\n\n'===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, please explain why it can\'t be generated. \n4. Please use the most relevant table(s). \n5. If the question has been asked and answered before, please repeat the answer exactly as it was given before. \n6. Ensure that the output SQL is SQLite-compliant and executable, and free of syntax errors. \n'"""},
    {"role": "user", "content": "Which tracks were purchased in the order with maximum tracks purchased?"}
    ],
    logit_bias= {2435: -100, 640: -100},
    max_tokens= 2000,
    temperature= 0, # Optional, Defaults to 1. Range: 0 to 2
    top_p= 1 # Optional, Defaults to 1. It is generally recommended to alter this or temperature but not both.
)

response = chat_completion.choices[0].message.content

PermissionDeniedError: <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;cloud&#46;olakrutrim&#46;com&#47;v1&#47;chat&#47;completions" on this server.<P>
Reference&#32;&#35;18&#46;57952317&#46;1725973340&#46;6056f98
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;57952317&#46;1725973340&#46;6056f98</P>
</BODY>
</HTML>